# Praktikum 2, Monte Carlo
## Math modeling of the atmosphere


## Computing average autoconversion

Autoconversion is the process by which small cloud droplets grow in size and become rain drops. It can be approximated as a function of two variables, $\chi$ and $N_c$:

$$A(\chi, N_c)=H(\chi){\chi}^{\alpha}{N_c}^{\beta}$$

Here $\chi$ is related to clound water mixing ratio, and $N_c$ is the number of cloud droplets. The expontent $\alpha$ is often taken to equal 2.47, and the exponent $\beta$ is often taken to equal -1.79. Therefore, if there is more cloud water or larger drops, rain forms more easily.

If we integrate over this autoconversion expression over a bivariate PDF that is normal in $\chi$ and lognormal in $N_c$, we find

TODO!

In [1]:
def exact_average_autoconversion_rate(my_n_c, my_chi,
                                      sigma_chi, sigma_n_c,
                                      r_chi_n_c, alpha, beta):
    import numpy as np
    from scipy.special import gamma, pbdv
    return (1 / (np.sqrt(2 * np.pi))) * (sigma_chi ** alpha) * np.exp((my_n_c * beta) + ((1/2) * sigma_n_c ** 2 * beta ** 2) - ((1/4) * ((my_chi / sigma_chi) + r_chi_n_c * sigma_n_c * beta) ** 2)) * gamma(alpha + 1) * pbdv(-(alpha + 1), -((my_chi/sigma_chi) + r_chi_n_c * sigma_n_c * beta))